In [1]:
import sys
from pathlib import Path

package_path = str(Path.cwd().parent)
if package_path not in sys.path:
    sys.path.append(package_path)

import time
import tqdm
import pickle
import numpy as np
import pandas as pd
import deepchem as dc

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate

import optuna

from tbprop.metrics import binary_classification_metrics
from tbprop.tree_based.preprocessing import VarianceThresholdPandas, StandardScalerPandas, \
        FeatureImportanceScoreThreshold, CorrelationThreshold
from tbprop.model_comparison import compare_models_optimizers_on_split

from rdkit.rdBase import BlockLogs

tqdm.tqdm.pandas()

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
/Users/vedang/miniconda3/envs/metal2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/Users/vedang/miniconda3/envs/metal2/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [2]:
DATA_DIR = "../data"

TRN_PATH = f"{DATA_DIR}/pk/pk_trn.csv"
VAL_PATH = f"{DATA_DIR}/pk/pk_val.csv"
TST_PATH = f"{DATA_DIR}/pk/pk_tst.csv"

MODE = 'bin_class'
RANDOM_SEED = 42

assert MODE in ('bin_class', 'reg', 'cat_class'), \
    f"MODE, '{MODE}' is invalid."

np.random.seed(RANDOM_SEED)

In [3]:
df_trn = pd.read_csv(TRN_PATH)
df_val = pd.read_csv(VAL_PATH)
df_tst = pd.read_csv(TST_PATH)

For simplicity, we'll remove all other columns from the dataset and assume it just has two columns, the column containing the SMILES string, and the column containing the label.

In [5]:
df_trn = df_trn[['smiles', 'auc_bin']]
df_val = df_val[['smiles', 'auc_bin']]
df_tst = df_tst[['smiles', 'auc_bin']]

In [6]:
df_trn.head()

,smiles,auc_bin
0,O=[N+]([O-])c1oc(C(=O)NC2CN(C(=O)c3[nH]c4c(c3)...,0
1,O=[N+]([O-])c1sc(/C=N/Nc2nc(NC(C)C)nc(NC(C)C)n...,1
2,Clc1ccc(CCC(=O)c2c(O)cc(O)cc2[O-])cc1,0
3,O=[N+]([O-])c1oc(CN=Nc2nc(Nc3ccccc3)cc(Nc3cccc...,0
4,S(=O)(=O)(Nc1cc2c(C)c(C(=O)N3CC4(C3)CCCCC4)[nH...,0


We'll use `deepchem` to add some features. Here we're using RDKit descriptors and circular fingerprints.

In [7]:
featurizers = {
    'circular': dc.feat.CircularFingerprint(size=2048, radius=4), 
    'rdkit': dc.feat.RDKitDescriptors()
}

def featurize(df, key, featurizer):
    feats = featurizer.featurize(df['smiles'])
    pd_feats = pd.DataFrame(feats, columns=[key + '_' + str(i+1) for i in range(feats.shape[1])])
    return pd.concat([df, pd_feats], axis=1)

with BlockLogs():
    for k, f in featurizers.items():
        print(f"Generating {k} fingerprints...")
        df_trn = featurize(df_trn, k, f)
        df_val = featurize(df_val, k, f)
        df_tst = featurize(df_tst, k, f)
    print("Done.")

print(f"Shape of trn set = {df_trn.shape}")
print(f"Shape of val set = {df_val.shape}")
print(f"Shape of tst set = {df_tst.shape}")

Generating circular fingerprints...
Generating rdkit fingerprints...
Done.
Shape of trn set = (114, 2260)
Shape of val set = (38, 2260)
Shape of tst set = (38, 2260)


In [9]:
df_trn.shape

(114, 2260)

#### Using the validation set for hyperparameter tuning

In [10]:
pipeline = Pipeline([
    ('var_thresh', VarianceThresholdPandas(threshold=.16)),
    ('corr_thresh', CorrelationThreshold(threshold=.95)),
    ('feat_imp_thresh', FeatureImportanceScoreThreshold(threshold=0., classifier='XGBoost', n_folds=3)),
    ('scaler', StandardScalerPandas())
])

X_trn, y_trn = df_trn.drop(['smiles', 'auc_bin'], axis=1), df_trn['auc_bin']
X_val, y_val = df_val.drop(['smiles', 'auc_bin'], axis=1), df_val['auc_bin']
X_tst, y_tst = df_tst.drop(['smiles', 'auc_bin'], axis=1), df_tst['auc_bin']

X_trn_proc = pipeline.fit_transform(X_trn, y_trn)
X_val_proc = pipeline.transform(X_val)
X_tst_proc = pipeline.transform(X_tst)

print(f"Final shape of trn set = {X_trn_proc.shape}")
print(f"Final shape of val set = {X_val_proc.shape}")
print(f"Final shape of tst set = {X_tst_proc.shape}")

Final shape of trn set = (114, 53)
Final shape of val set = (38, 53)
Final shape of tst set = (38, 53)


In [11]:
models = [RandomForestClassifier, XGBClassifier, LGBMClassifier, CatBoostClassifier]

trained_models, test_metrics = \
    compare_models_optimizers_on_split(models, 
                                       X_trn_proc, y_trn, 
                                       X_tst_proc, y_tst, 
                                       X_val=X_val_proc, y_val=y_val,
                                       random_state=RANDOM_SEED,
                                       random_seed=RANDOM_SEED,
                                       max_evals=20,
                                       val_mode='fixed_split',
                                       pipeline_suffix='P1',
                                       optimizer_types=['hyperopt', 'optuna', 'random_search'])

Training models using 'hyperopt'.

Optimizing: RandomForestClassifier.
100%|██████████| 20/20 [00:09<00:00,  2.13trial/s, best loss: -0.89197]
Optimizing: XGBClassifier.
100%|██████████| 20/20 [00:01<00:00, 14.74trial/s, best loss: -0.74377]
Optimizing: LGBMClassifier.
100%|██████████| 20/20 [00:00<00:00, 32.69trial/s, best loss: -0.8642699999999999]
Optimizing: CatBoostClassifier.
100%|██████████| 20/20 [00:35<00:00,  1.75s/trial, best loss: -0.83934]

Model: RandomForestClassifier | Test AUROC: 0.89196675900277, AP: 0.811636698309747, Max F1: 0.8780487804878049
Model: XGBClassifier | Test AUROC: 0.8670360110803323, AP: 0.8955950199583473, Max F1: 0.7878787878787878
Model: LGBMClassifier | Test AUROC: 0.9030470914127424, AP: 0.9093003212365276, Max F1: 0.8717948717948718
Model: CatBoostClassifier | Test AUROC: 0.9141274238227147, AP: 0.9061588983118938, Max F1: 0.8837209302325582

Optimizer time = 59.852 s.

Training models using 'optuna'.

Optimizing: RandomForestClassifier.


Best trial: 14. Best value: -0.93906: 100%|██████████| 20/20 [00:03<00:00,  5.24it/s]


Optimizing: XGBClassifier.


Best trial: 7. Best value: -0.94737: 100%|██████████| 20/20 [00:09<00:00,  2.21it/s]


Optimizing: LGBMClassifier.


Best trial: 9. Best value: -0.93906: 100%|██████████| 20/20 [00:00<00:00, 24.45it/s]


Optimizing: CatBoostClassifier.


Best trial: 11. Best value: -0.98615: 100%|██████████| 20/20 [00:21<00:00,  1.10s/it]



Model: RandomForestClassifier | Test AUROC: 0.8975069252077562, AP: 0.8926143879023949, Max F1: 0.8500000000000001
Model: XGBClassifier | Test AUROC: 0.8975069252077562, AP: 0.879500199754484, Max F1: 0.8500000000000001
Model: LGBMClassifier | Test AUROC: 0.8947368421052632, AP: 0.9099008610475936, Max F1: 0.8205128205128205
Model: CatBoostClassifier | Test AUROC: 0.9252077562326869, AP: 0.9138707353877629, Max F1: 0.9

Optimizer time = 59.474 s.

Training models using 'random_search'.

Optimizing: RandomForestClassifier.
Fitting 5 folds for each of 5 candidates, totalling 25 fits
  Time taken = 4s
Optimizing: XGBClassifier.
Fitting 5 folds for each of 5 candidates, totalling 25 fits
  Time taken = 1s
Optimizing: LGBMClassifier.
Fitting 5 folds for each of 5 candidates, totalling 25 fits
  Time taken = 0s
Optimizing: CatBoostClassifier.
Fitting 5 folds for each of 5 candidates, totalling 25 fits
  Time taken = 18s

Model: RandomForestClassifier | Test AUROC: 0.8836565096952909, AP: 0.

Following this you can save the model, results, and pipeline. I've commented it out so as not to overwrite files.

In [10]:

# Saving data
# with open(f"{DATA_DIR}/trained_models.pkl", 'wb') as f:
#     pickle.dump(
#         {
#             'trained_models': trained_models,
#             'test_metrics': test_metrics,
#             'pipeline': pipeline
#         },
#         f
#     )

# Reading back data
# with open(f"{DATA_DIR}/trained_models.pkl", 'rb') as f:
#     metadata = pickle.load(f)

#### Using k-fold cross validation for hyperparameter tuning

Alternatively, you can use k-fold CV for hyperparameter tuning. In this case, we don't need a validation set so we mix it back with the training set.

In [21]:
X_trn, y_trn = df_trn.drop(['smiles', 'auc_bin'], axis=1), df_trn['auc_bin']
X_val, y_val = df_val.drop(['smiles', 'auc_bin'], axis=1), df_val['auc_bin']
X_tst, y_tst = df_tst.drop(['smiles', 'auc_bin'], axis=1), df_tst['auc_bin']

train_X, train_y = pd.concat([X_trn, X_val]), pd.concat([y_trn, y_val])

During k-fold CV, for each validation split, we need to call `fit_transform()` on the training data and `transform()` on the validation data. This means that the entire pipeline (and not just the model) needs to be fit in each of the k iterations of k-fold CV. Scikit-learn allows us to wrap the `pipeline` in the `cross_validate` function to achieve this. At the end of the pipeline, we'll add our classifier, and given a new suggestion for the classifier parameters, we should be able to initialize the entire pipeline with those parameters. This is done in the function below.

In the future, even pipeline parameters can be set through hyperparameter optimization.

In [ ]:
def build_pipeline(params):
    pipeline = Pipeline([
        ('var_thresh', VarianceThresholdPandas(threshold=.16)),
        ('corr_thresh', CorrelationThreshold(threshold=.95)),
        ('feat_imp_thresh', FeatureImportanceScoreThreshold(threshold=0., classifier='XGBoost', n_folds=3)),
        ('scaler', StandardScalerPandas()),
        ('classifier', XGBClassifier(**params))
    ])
    return pipeline

def build_params(trial: optuna.trial.Trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 1),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.01),
        'subsample': trial.suggest_float('subsample', 0.1, 1)
    }
    return params

The next step is to build an "optimizer" class. Building a class is not necessary, you can simply create a study, an objective function, and a search space and use it, but building a class is good practice from an object-oriented perspective (encapsulation of data and interfacing). Overall the steps here are:

1. Initialize a study and define an evaluation metric.
2. Define a search space (`_build_params`).
3. Define an objective function (`objective`).
4. Run the study until either `n_trials` or `timeout` number of seconds are reached.

In [32]:
class OptunaOptimizer:
    def __init__(self, X_trn, y_trn, eval_metric='roc_auc', n_folds=5):
        """ Optuna optimizer. """
        self.X_trn = X_trn
        self.y_trn = y_trn
        self.eval_metric = eval_metric
        self.n_folds = n_folds

    def objective(self, trial):
        """ Objective function for Optuna. """
        params = build_params(trial)
        clf = build_pipeline(params)
        res = cross_validate(clf, self.X_trn, self.y_trn, scoring=self.eval_metric, cv=self.n_folds)
        return res['test_score'].mean()
    
    def _name_study(self):
        """ Generate a unique name for the study. """
        return f'optuna_{time.strftime("%Y%m%d-%H%M%S")}'

    def optimize(self, n_trials=100, timeout=600, show_progress_bar=True):
        """ Run the optimization. """
        study = optuna.create_study(
            study_name=self._name_study(),
            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
        )

        study.optimize(self.objective, 
                       n_trials=n_trials, 
                       timeout=timeout, 
                       show_progress_bar=show_progress_bar)
        
        return study.best_trial

In [35]:
optimizer = OptunaOptimizer(X_trn=train_X, y_trn=train_y, eval_metric="roc_auc", n_folds=5)
best_trial = optimizer.optimize(n_trials=10)

[I 2024-05-09 10:51:09,938] A new study created in memory with name: optuna_20240509-105109
Best trial: 0. Best value: 0.788944:  10%|█         | 1/10 [00:11<01:47, 11.93s/it, 11.93/600 seconds]

[I 2024-05-09 10:51:21,871] Trial 0 finished with value: 0.7889444444444444 and parameters: {'n_estimators': 138, 'max_depth': 13, 'min_child_weight': 0.3765168195645399, 'learning_rate': 0.006804991368905178, 'subsample': 0.8907299462217086}. Best is trial 0 with value: 0.7889444444444444.


Best trial: 1. Best value: 0.798389:  20%|██        | 2/10 [00:23<01:31, 11.43s/it, 23.01/600 seconds]

[I 2024-05-09 10:51:32,948] Trial 1 finished with value: 0.7983888888888889 and parameters: {'n_estimators': 160, 'max_depth': 6, 'min_child_weight': 0.8108615632945625, 'learning_rate': 0.004774108094506331, 'subsample': 0.6533436118206156}. Best is trial 1 with value: 0.7983888888888889.


Best trial: 2. Best value: 0.805333:  30%|███       | 3/10 [00:33<01:18, 11.16s/it, 33.86/600 seconds]

...

[I 2024-05-09 10:52:43,482] Trial 7 finished with value: 0.7949444444444445 and parameters: {'n_estimators': 378, 'max_depth': 12, 'min_child_weight': 0.19875781439395948, 'learning_rate': 0.0036547254082179175, 'subsample': 0.9374497812267503}. Best is trial 3 with value: 0.8156111111111111.


Best trial: 3. Best value: 0.815611:  90%|█████████ | 9/10 [01:44<00:11, 11.45s/it, 104.06/600 seconds]

[I 2024-05-09 10:52:53,996] Trial 8 finished with value: 0.6907222222222222 and parameters: {'n_estimators': 392, 'max_depth': 11, 'min_child_weight': 0.7283640112669343, 'learning_rate': 0.004970358958154457, 'subsample': 0.10685813231177621}. Best is trial 3 with value: 0.8156111111111111.


Best trial: 9. Best value: 0.817111: 100%|██████████| 10/10 [01:55<00:00, 11.58s/it, 115.78/600 seconds]

[I 2024-05-09 10:53:05,720] Trial 9 finished with value: 0.8171111111111113 and parameters: {'n_estimators': 454, 'max_depth': 11, 'min_child_weight': 0.6772319601101815, 'learning_rate': 0.004773750316364623, 'subsample': 0.7488515680732578}. Best is trial 9 with value: 0.8171111111111113.


In [37]:
# Best parameters found for the pipeline
best_trial.params

{'n_estimators': 454,
 'max_depth': 11,
 'min_child_weight': 0.6772319601101815,
 'learning_rate': 0.004773750316364623,
 'subsample': 0.7488515680732578}

Test the performance of the "best" pipeline by training it on the entire training (trn + val) set once and then testing it on the test set.

In [53]:
def test_performance(params, X_trn, y_trn, X_tst, y_tst):
    best_model = build_pipeline(params)
    best_model.fit(X_trn, y_trn)
    y_score = best_model.predict_proba(X_tst)[:,1]
    y_pred = best_model.predict(X_tst)
    return binary_classification_metrics(y_tst, y_pred, y_score)

k_fold_test_metrics = test_performance(best_trial.params, train_X, train_y, X_tst, y_tst)
k_fold_test_metrics

,Metric,Value
0,n_pos,15.000
1,accuracy,78.947
2,balanced_accuracy,78.947
3,auroc,88.643
4,ap,87.370
5,precision,86.667
6,recall,68.421
7,f1_score,76.471


Train one final time

In [ ]:
best_model = build_pipeline(best_trial.params)
best_model.fit(pd.concat([train_X, X_tst]), pd.concat([train_y, y_tst]))

### Save model
# with open(f"{DATA_DIR}/best_pipeline_pk.pkl", "wb") as f:
#     pickle.dump({
#         "test_metrics": k_fold_test_metrics,
#         "trained_pipeline": best_model
#     }, f)

You should save the best pipeline and metrics for inference on other datasets.